In [1]:
import pandas as pd
DATA_DIR = "/home/idies/workspace/Storage/colemanzhang98/persistent/git_repo/ml-research/datasets/"
df = pd.read_csv(DATA_DIR + 'data.zip', compression='zip', parse_dates=['DATE', 'datadate'])
df.columns = map(str.lower, df.columns)
sec = ['date', 'datadate', 'permno', 'gvkey', 'sic2']
firm_char = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 
             'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt', 'cashpr', 'cf', 
             'cfp', 'cfp_ia', 'chadv', 'chato', 'chatoia', 'chcsho', 'chdrc', 
             'chempia', 'chinv', 'chmom', 'chobklg', 'chpm', 'chpmia', 'chtx', 
             'cinvest', 'conv', 'convind', 'count', 'credrat', 'credrat_dwn', 
             'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'eamonth', 
             'ear', 'egr', 'ep', 'gma', 'grcapx', 'grgw', 'grltnoa', 'herf', 
             'hire', 'idiovol', 'ill', 'indmom', 'invest', 'ipo', 'lev', 'lgr', 
             'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mve', 
             'mve_ia', 'nincr', 'obklg', 'operprof', 'orgcap', 'pchcapx', 
             'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 
             'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 
             'pchsaleinv', 'pctacc', 'prccq', 'pricedelay', 'ps', 'quick', 
             'rd', 'rd_mve', 'rd_sale', 'rdbias', 'rdq', 'realestate', 
             'retcons_neg', 'retcons_pos', 'retvol', 'roaq', 'roavol', 'roe', 
             'roeq', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 
             'secured', 'securedind', 'sgr', 'sgrvol', 'sin', 'sp', 'spi', 
             'spii', 'std_dolvol', 'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 
             'turn', 'wogw', 'zerotrade'] # 119 firm characteristics

crsp_list = ['exchcd', 'dlret', 'dlstcd', 'vol', 'shrout', 'prc', 'ewret', 'ret']
import numpy as np
df_cols = np.append(np.append(np.asarray(sec), firm_char), crsp_list)
df = df[df_cols]
df = df.loc[~df.duplicated(subset=['date', 'gvkey'])]

In [2]:
# Add Research Qutient & Tobin's Q; shift yearly updated characteristic by 6 months
q = pd.read_csv(DATA_DIR + "q.zip", index_col = 0, parse_dates=[1, 3], compression="zip")
rq = pd.read_csv(DATA_DIR + "rq.zip", index_col = 0, parse_dates=[1], compression="zip")
q['gvkey'] = q['gvkey'].astype(pd.Int64Dtype())
q['fyear'] = q['fyear'].astype(pd.Int64Dtype())
rq['gvkey'] = rq['gvkey'].astype(pd.Int64Dtype())
rq['fyear'] = rq['fyear'].astype(pd.Int64Dtype())
pd.set_option("display.max_rows", 1000)

/home/idies/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# q.insert(1, 'date', q['datadate'] + pd.DateOffset(months=6)) # shift annual characteristics by 6 months lag
# def q_to_month(group):
#     last_yr = group.datadate.max()
#     group = group.append(pd.Series({'datadate': last_yr + pd.DateOffset(years=1), 
#                                     'date': last_yr + pd.DateOffset(years=1) + pd.DateOffset(months=6)}), 
#                          ignore_index=True)
#     group = group.set_index('date')
#     group.index += pd.offsets.MonthEnd(0)
#     group = group.asfreq(freq='m').ffill(limit=12)
#     group = group.iloc[:-1]
#     return group
# q_ = q.groupby('gvkey', as_index=False).apply(q_to_month).reset_index().drop("level_0", axis=1)
# q_['gvkey'] = q_['gvkey'].astype(pd.Int64Dtype())
# q_['fyear'] = q_['fyear'].astype(pd.Int64Dtype())

In [0]:
# rq = rq[~rq.duplicated(subset=['gvkey', 'date'])]
# rq.insert(1, 'date', pd.to_datetime(rq['fyear'].astype(str) + '1231') + pd.DateOffset(months=6))
# def rq_to_month(group):
#     last_yr = group.date.max()
#     group = group.append(pd.Series({'date': last_yr + pd.DateOffset(years=1)}), 
#                          ignore_index=True)
#     group = group.set_index('date')
#     group.index += pd.offsets.MonthEnd(0)
#     group = group.asfreq(freq='m').ffill(limit=12)
#     group = group.iloc[:-1]
#     return group
# rq_ = rq.groupby('gvkey', as_index=False).apply(rq_to_month).reset_index().drop("level_0", axis=1)
# rq_['gvkey'] = rq_['gvkey'].astype(pd.Int64Dtype())
# rq_['fyear'] = rq_['fyear'].astype(pd.Int64Dtype())
# rq_.head()

In [3]:
q.insert(1, 'yyyymm', q['date'].dt.strftime('%Y%m').astype(int))
rq.insert(1, 'yyyymm', rq['date'].dt.strftime('%Y%m').astype(int))

In [5]:
macro = pd.read_excel(DATA_DIR + 'macroecon.xlsx', parse_dates=['yyyymm'])

In [6]:
macro['dp'] = np.log(macro['D12']) - np.log(macro['Index'])
macro['ep'] = np.log(macro['E12']) - np.log(macro['Index'])
macro['bm'] = macro['b/m']
macro['tms'] = macro['lty'] - macro['tbl']
macro['dfy'] = macro['BAA'] - macro['AAA']
macro = macro.loc[macro['yyyymm']>=196001, ['yyyymm', 'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar']]

In [7]:
df.insert(1, 'yyyymm', df['date'].dt.strftime('%Y%m').astype(int))

In [8]:
# Merge firm char, CRSP with rq, q, and macro economic factors
from functools import reduce
q_reduced = q.loc[q['gvkey'].notnull(), ['yyyymm', 'gvkey', 'q_tot']].copy()
rq_reduced = rq.loc[rq['gvkey'].notnull(), ['yyyymm', 'gvkey', 'aggbeta_lxrd']].copy()
dataframes = [df, q_reduced, rq_reduced]
df_ = reduce(lambda left, right: pd.merge(left, right, on=['gvkey', 'yyyymm'],
                                          how='left'), dataframes)
df2 = pd.merge(df_, macro, how='left', on='yyyymm', suffixes=('', '_macro'))

In [ ]:
del df_
del df
del q
del rq
del q_reduced
del rq_reduced

In [9]:
# Reorder columns
macro_cols = ['dp', 'ep_macro', 'bm_macro', 'ntis', 'tbl', 'tms', 'dfy', 'svar']
cols = [np.asarray(sec), firm_char, macro_cols, np.asarray(['q_tot', 'aggbeta_lxrd']), crsp_list]
df_cols = reduce(lambda left, right: np.append(left, right), cols)
df2 = df2[df_cols]

In [ ]:
# Create column for security returns at T+1
df2.insert(1, 'yyyymmdd', df2['date'].dt.strftime('%Y%m%d').astype(int))
dates = df2['yyyymmdd'].unique()
x = np.sort(dates)
x = np.append(x, 20190131)
y = df2['yyyymmdd'].values
df2.insert(2, 'yyyymmdd+1', x[np.where(y.reshape(y.size, 1) == x)[1]+1])

exp_df = pd.merge(df2, df2[['gvkey', 'yyyymmdd', 'ewret', 'ret']], 
                  left_on=['gvkey', 'yyyymmdd+1'], right_on=['gvkey', 'yyyymmdd'], how='left', suffixes=("", "+1m"))

In [ ]:
del exp_df['yyyymmdd']
del exp_df['yyyymmdd+1']
del exp_df['yyyymmdd+1m']
exp_df = exp_df[exp_df['date'] > pd.to_datetime("1965-05-31")] # start from month with sufficient observations
del df2

In [ ]:
firm_char_ = np.sort(np.append(firm_char, np.asarray(['q_tot', 'aggbeta_lxrd'])))
exp_df = exp_df.sort_values(by=['date', 'gvkey'])

# Rank firm characteristics within period cross-secionally and normalize to [-1, 1]
def char_rank(group):
    group[firm_char_] = group[firm_char_].rank(pct=True) * 2 - 1
    return group

for char in firm_char_:
    exp_df.insert(exp_df.columns.get_loc(char) + 1, f"{char}_missing", np.nan)
char_missing = np.array([f"{char}_missing" for char in firm_char_])

# Fill by cross-secion median by period
def fill_missing(group):
    group[char_missing] = group[firm_char_].isnull().astype(np.int)
    group[firm_char_] = group[firm_char_].fillna(group[firm_char_].median())
    return group

# Create label based on return at T+1 (performance ranking)
def add_label(group):
    group = group[group['ret+1m'].notnull()]
    group['label'] = group['ret+1m'].rank(ascending=False)
    return group

exp_df = exp_df.groupby('date').apply(char_rank)
exp_df = exp_df.groupby('date').apply(fill_missing)
exp_df = exp_df.groupby('date').apply(add_label)

In [4]:
firm_char_ = np.sort(np.append(firm_char, np.asarray(['q_tot', 'aggbeta_lxrd'])))
a = exp_df[firm_char_].to_numpy(copy=False) # d1.values on older pandas versions
b = exp_df[macro_cols].to_numpy(copy=False)

# Create interactions between macroeconomic factors and firm characteristics
import numexpr as ne
out = ne.evaluate('a3D*b3D',{'a3D':a[:,:,None],'b3D':b[:,None]}).reshape(len(a),-1)
df_out = pd.DataFrame(out)
df_out.columns = [f"{i}*{j}" for i in firm_char_ for j in macro_cols]
df_out.index = exp_df.index
exp_df = pd.concat([exp_df, df_out], axis=1)

cols = [np.asarray(sec), firm_char_, char_missing, df_out.columns.values, 
        crsp_list, np.asarray(['ewret+1m', 'ret+1m', 'label'])]
df_cols = reduce(lambda left, right: np.append(left, right), cols)
exp_df = exp_df[df_cols]
exp_df = exp_df[~((exp_df['dlstcd'].notnull()) & (exp_df['dlstcd']>400))] # drop delisted companies

NameError: name 'np' is not defined

In [ ]:
exp_df.head(100).to_csv("/home/idies/workspace/Storage/colemanzhang98/persistent/scratch/header_with_label.csv", index=False)

In [0]:
# exp_df.to_csv("/home/idies/workspace/Temporary/colemanzhang98/scratch/data_labeled.zip", compression='zip', index=False)

In [ ]:
# Perform PLS
from sklearn.cross_decomposition import PLSRegression
pls2 = PLSRegression(n_components=100)
exp_df = pd.get_dummies(exp_df, dummy_na=True, columns=['sic2'], drop_first=True)
dummy_cols = np.asarray([[col for col in exp_df.columns if "sic2_" in col]])
cols = [dummy_cols, firm_char_, char_missing, df_out.columns.values]
feature_cols = reduce(lambda left, right: np.append(left, right), cols)
train = exp_df.loc[exp_df['date'] <= pd.to_datetime("1981-06-30"), :]
X = train[feature_cols].fillna(0)
y = train['label']
pls2.fit(X, y)
P = pls2.x_loadings_
df_P = pd.DataFrame(P, index=X.columns, columns=[f"comp_{i}" for i in range(1, 101)])
df_P.to_csv("/home/idies/workspace/Storage/colemanzhang98/persistent/scratch/X_loadings.csv")
T = pls2.x_scores_
df_T = pd.DataFrame(T, index=train.index, columns=[f"comp_{i}" for i in range(1, 101)])
output = pd.concat([train[sec], df_T, train[['ret', 'ret+1m', 'label']]], axis=1)
output.to_csv("/home/idies/workspace/Temporary/colemanzhang98/scratch/reduced.csv.zip", compression='zip')